In [1]:
import psycopg2
import geopandas as gpd

import rasterio
from rasterio.plot import show
from rasterio.io import MemoryFile
from io import BytesIO
from shapely.geometry import shape
import pandas as pd
import numpy as np
import json

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import LogColorMapper, ColorBar, LinearColorMapper
from bokeh.layouts import column
from bokeh.palettes import Viridis256
output_notebook()

Loading BokehJS ...

## Test the Postgres query to ensure that the raster is clipped by the polygon geometry as expected

In [2]:
# Postgres connection params
conn_params = {
    'dbname': 'basins',
    'user': 'postgres',
    'password': 'pgpass',
    'host': 'localhost',
    'port': '5432'
}


In [3]:
def basic_query(q):
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()
    # Execute the query and fetch the results
    cursor.execute(q)
    results = cursor.fetchall()
    # Close the cursor and connection
    cursor.close()
    conn.close()
    return results


In [4]:
def plot_clipped_rasters_and_polygons(conn_params, basins_table, raster_table, ids):
    # Connect to the PostgreSQL database
    # Create the SQL query for selected basins
    basins_query = f"""
    WITH selected_basins AS (
        SELECT id, basin, drainage_area_km2
        FROM {basins_table}
        WHERE id IN ({','.join(map(str, ids))})
    ), clipped_rasters AS (
        SELECT
            b.id, 
            b.drainage_area_km2,
            ST_AsGeoJSON(b.basin) AS basin_geojson,
            ST_AsGDALRaster(ST_Clip(r.rast, 1, b.basin, TRUE), 'GTiff', ARRAY['QUALITY=100'], 3005) AS rast_clipped
        FROM 
            {raster_table} r
        JOIN selected_basins b 
        ON ST_Intersects(ST_ConvexHull(r.rast), b.basin)
    )
    SELECT * FROM clipped_rasters;
    """
    results = basic_query(basins_query)
    
    # Initialize lists to hold GeoDataFrame and rasters
    polygons = []
    rasters = []
    ids = []
    areas = []
    for row in results:
        ids.append(row[0])
        p = shape(json.loads(row[2]))
        a = row[1]
        areas.append(a)
        polygons.append(p)
        rasters.append(row[3])

    # Create a GeoDataFrame from the polygons
    gdf = gpd.GeoDataFrame({'id': ids, 'geometry': polygons, 'drainage_area_km2': areas}, crs='EPSG:3005')
    gdf['area_check'] = gdf.geometry.area / 1e6
   
    # Initialize Bokeh plot
    p = figure(title="Clipped Rasters and Polygons", match_aspect=True)
    p.grid.grid_line_color = None

    # Plot polygons
    for geom, area, ac in zip(gdf['geometry'], gdf['drainage_area_km2'], gdf['area_check']):
        if geom.geom_type == 'Polygon':
            x, y = geom.exterior.xy
            p.patch(x, y, fill_alpha=0.5, line_width=2, legend_label=f'{area:.0f}/{ac:.0f}')
        elif geom.geom_type == 'MultiPolygon':
            for subgeom in geom:
                x, y = subgeom.exterior.xy
                p.patch(x, y, fill_alpha=0.5, line_width=2)

    # Plot rasters
    for idx, raster_data in enumerate(rasters):
        with MemoryFile(raster_data) as memfile:
            with memfile.open() as src:
                # Read raster data
                raster = src.read(1)
                raster_extent = (src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top)
                
                # Mask no data values
                raster = np.ma.masked_equal(raster, src.nodata).astype(float)

                # Flip raster vertically for correct orientation
                raster = np.flipud(raster)
                
                # Create color mapper
                color_mapper = LinearColorMapper(palette=Viridis256, low=raster.min(), high=raster.max())
                
                # Add raster to plot
                p.image(image=[raster], x=raster_extent[0], y=raster_extent[2], dw=raster_extent[1]-raster_extent[0], dh=raster_extent[3]-raster_extent[2], color_mapper=color_mapper, alpha=0.5)

    p.xaxis.axis_label ='Longitude'
    p.yaxis.axis_label = 'Latitude'
    show(p)

In [5]:
plot_clipped_rasters_and_polygons(conn_params, 'basins_schema.basin_attributes', 'basins_schema.nalcms_2010', [1050544,685317])


In [6]:
q = """
SELECT id, drainage_area_km2
FROM basins_schema.basin_attributes
LIMIT 1;
"""
da = basic_query(q)
da

[(143465, 11.307232582556345)]

In [7]:
q = """
SELECT column_name
FROM information_schema.columns
WHERE table_schema = 'basins_schema'
  AND table_name = 'basin_attributes';
"""
columns = basic_query(q)
columns = [c[0] for c in columns]
print(columns)
lu_cols = [c for c in columns if c.startswith('land_use_')]
lu_cols_str = ', '.join(lu_cols)

['id', 'centroid', 'pour_pt', 'basin', 'fid', 'area', 'drainage_area_km2', 'acc_polygon', 'ppt_acc', 'ppt_lon_m_3005', 'ppt_lat_m_3005', 'elevation_m', 'aspect_deg', 'slope_deg', 'region_code', 'perimeter_km', 'gravelius', 'centroid_x', 'centroid_y', 'prcp', 'tmin', 'tmax', 'vp', 'swe', 'srad', 'high_prcp_freq', 'low_prcp_freq', 'high_prcp_duration', 'low_prcp_duration', 'land_use_forest_frac_2010', 'land_use_forest_frac_2015', 'land_use_forest_frac_2020', 'land_use_shrubs_frac_2010', 'land_use_shrubs_frac_2015', 'land_use_shrubs_frac_2020', 'land_use_grass_frac_2010', 'land_use_grass_frac_2015', 'land_use_grass_frac_2020', 'land_use_wetland_frac_2010', 'land_use_wetland_frac_2015', 'land_use_wetland_frac_2020', 'land_use_crops_frac_2010', 'land_use_crops_frac_2015', 'land_use_crops_frac_2020', 'land_use_urban_frac_2010', 'land_use_urban_frac_2015', 'land_use_urban_frac_2020', 'land_use_water_frac_2010', 'land_use_water_frac_2015', 'land_use_water_frac_2020', 'land_use_snow_ice_frac_20

In [8]:
q = f"""
SELECT {lu_cols_str}
FROM basins_schema.basin_attributes
LIMIT 1;
"""
ex = basic_query(q)

In [9]:
for c in lu_cols:
    q = f"""
    SELECT COUNT(*)
    FROM basins_schema.basin_attributes
    WHERE {c} IS NOT NULL;
    """
    non_null = basic_query(q)
    print(c, ' - ', non_null)

land_use_forest_frac_2010  -  [(1264917,)]
land_use_forest_frac_2015  -  [(1264917,)]
land_use_forest_frac_2020  -  [(1264917,)]
land_use_shrubs_frac_2010  -  [(1264917,)]
land_use_shrubs_frac_2015  -  [(1264917,)]
land_use_shrubs_frac_2020  -  [(1264917,)]
land_use_grass_frac_2010  -  [(1264917,)]
land_use_grass_frac_2015  -  [(1264917,)]
land_use_grass_frac_2020  -  [(1264917,)]
land_use_wetland_frac_2010  -  [(1264917,)]
land_use_wetland_frac_2015  -  [(1264917,)]
land_use_wetland_frac_2020  -  [(1264917,)]
land_use_crops_frac_2010  -  [(1264917,)]
land_use_crops_frac_2015  -  [(1264917,)]
land_use_crops_frac_2020  -  [(1264917,)]
land_use_urban_frac_2010  -  [(1264917,)]
land_use_urban_frac_2015  -  [(1264917,)]
land_use_urban_frac_2020  -  [(1264917,)]
land_use_water_frac_2010  -  [(1264917,)]
land_use_water_frac_2015  -  [(1264917,)]
land_use_water_frac_2020  -  [(1264917,)]
land_use_snow_ice_frac_2010  -  [(1264917,)]
land_use_snow_ice_frac_2015  -  [(1264917,)]
land_use_snow_ic